In [1]:
from numba import cuda
import numpy as np
import matplotlib.pyplot as plt
import math

## Example：计算数组中每一个元素的平方值

In [2]:
@cuda.jit # kernel function
def square_kernel(INPUT,OUTPUT,rows,cols):
    i,j = cuda.grid(2)
    if(i<rows and j<cols):
        OUTPUT[i,j]=INPUT[i,j]*INPUT[i,j]

def square(M):
    rows=M.shape[0]
    cols=M.shape[1]
    
    device_INPUT = cuda.to_device(M)
    device_OUTPUT = cuda.device_array_like(device_INPUT)
    # Configure the blocks
    threadsperblock = (16, 16)
    blockspergrid_x = int(math.ceil(rows / threadsperblock[0]))
    blockspergrid_y = int(math.ceil(cols / threadsperblock[0]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)
    square_kernel[blockspergrid, threadsperblock](device_INPUT,device_OUTPUT,rows,cols)
    return device_OUTPUT.copy_to_host()

In [3]:
M=np.random.randn(3,2)
print(M)

[[ 0.1646849  -0.4861291 ]
 [ 0.26488708 -0.78395807]
 [-1.49390283  0.15972329]]


In [4]:
print(square(M),end='\n\n')
print(M**2) # use numpy to computer square
np.alltrue(M**2==square(M))

[[0.02712112 0.2363215 ]
 [0.07016517 0.61459026]
 [2.23174566 0.02551153]]

[[0.02712112 0.2363215 ]
 [0.07016517 0.61459026]
 [2.23174566 0.02551153]]


True